In [137]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn import preprocessing

In [138]:
train_file = pd.read_csv(r'D:\Datasets\titanic\train.csv')
test_file = pd.read_csv(r'D:\Datasets\titanic\test.csv')

In [139]:
sc = preprocessing.MinMaxScaler()
cols = train_file.columns
t_cols = test_file.columns
x = train_file[cols[2:]]
y = train_file[cols[1]]
x_t = test_file[t_cols[1:]]
y = y.values
y.resize(891, 1)
x.fillna(x.median(), inplace=True)
x_t.fillna(x.median(), inplace=True)
x = pd.get_dummies(x, drop_first=True, dummy_na=True)
x_t = pd.get_dummies(x_t, drop_first=True, dummy_na=True)
x = sc.fit_transform(x)
x_t = sc.fit_transform(x_t)
x1 = pd.DataFrame(x)
x2 = pd.DataFrame(x_t)
q = list(set(x1.columns) - set(x2.columns))
for i in q:
    x2[i] = 0
x_t = x2.values

C:\Python\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [140]:
X_train = Variable(torch.from_numpy(x).float())
y_train = Variable(torch.from_numpy(y).float())
X_test = Variable(torch.from_numpy(x_t).float())
print(X_test.shape)
print(y_train.shape)

torch.Size([418, 1729])
torch.Size([891, 1])


In [141]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(1729, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 200)
        self.fc4 = nn.Linear(200, 128)
        self.fc5 = nn.Linear(128, 1)
        self.drop = nn.Dropout(0.2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(F.relu(self.fc2(x)))
        x = self.drop(F.relu(self.fc3(x)))
        x = self.drop(F.relu(self.fc4(x)))
        x = F.relu(self.fc5(x))
        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = Model().cuda()

In [142]:
loss_fn = nn.MSELoss()
opt = torch.optim.Adam(net.parameters())

In [143]:
epochs = 3000
for epoch in range(epochs):
    preds = net(Variable(X_train.cuda()))
    opt.zero_grad()
    loss = loss_fn(preds, Variable(y_train.cuda()))
    loss.backward()
    opt.step()
    if epoch % 100 == 0:
        print(f"Loss: {loss.item()}")

Loss: 0.3771471679210663
Loss: 0.004925186280161142
Loss: 0.0035371030680835247
Loss: 0.002710781991481781
Loss: 0.0022823428735136986
Loss: 0.002177431248128414
Loss: 0.002443597884848714
Loss: 0.0020697470754384995
Loss: 0.002111799782142043
Loss: 0.0017963751452043653
Loss: 0.0018599667819216847
Loss: 0.001871851272881031
Loss: 0.0014297142624855042
Loss: 0.001431957702152431
Loss: 0.001452346332371235
Loss: 0.0013505842071026564
Loss: 0.001282847486436367
Loss: 0.0011040413519367576
Loss: 0.001258795615285635
Loss: 0.0011457244399935007
Loss: 0.001041362644173205
Loss: 0.001173241063952446
Loss: 0.0009382744319736958
Loss: 0.0010691243223845959
Loss: 0.0010044416412711143
Loss: 0.0010017758468165994
Loss: 0.000976809300482273
Loss: 0.0008898120140656829
Loss: 0.0008368542767129838
Loss: 0.0007879924378357828


In [144]:
pred = net(X_test.cuda())

In [145]:
a =(pred<0.5).int()
a = a.cpu().numpy()

In [146]:
p = []
for i in a:
    for j in i:
        p.append(j) 
p

[0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,


In [147]:
dat = list()
dat2 = list()
for i in p:
    n = {}
    n["Survived"] = i
    dat.append(n)

for j in test_file["PassengerId"]:
    k = {}
    k["PassengerId"] = j
    dat2.append(k)
    
dat

[{'Survived': 0},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 0},
 {'Survived': 0},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survived': 1},
 {'Survive

In [148]:
a = pd.DataFrame.from_dict(dat)
b = pd.DataFrame.from_dict(dat2)
c = pd.concat([b, a], axis=1)
c.to_csv("titan.csv")